#  Database Testing
- toc: false
- badges: false
- comments: true
- categories: [Unit2]

In [ ]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

class ChessUsers(db.Model):
    __tablename__ = 'chess_users'
    
    # Define the Users schema
    gid = db.Column(db.Integer, primary_key=True)
    uid1 = db.Column(db.Integer, nullable=False)
    name1 = db.Column(db.String(255), unique=False, nullable=False)
    uid2 = db.Column(db.Integer, nullable=False)
    name2 = db.Column(db.String(255), unique=False, nullable=False)
    moves = db.Column(db.BLOB, unique = False, nullable=False)
    # Defines a relationship between User record and Notes table, one-to-many (one user to many notes)
    # notes = db.relationship("Notes", cascade='all, delete', backref='users', lazy=True)

    # constructor of a User object, initializes of instance variables within object
    def __init__(self, name='', uid="0", password="null", dob="11-11-1111", games=""):
        self.gid = self.make_gid()
        self.name = name
        self.dob = dob
        self.games = ""

    # returns a string representation of object, similar to java toString()
    def __repr__(self):
        return "Users(" + str(self.uid) + "," + self.name + "," + str(self.dob) +  str(self.games) + ")"

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from Users(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            print("hello")
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "uid": self.uid,
            "name": self.name,
            "password": self.password,
            "dob": self.dob,
            "games": self.games
        }

    # CRUD update: updates users name, password, phone
    # returns self
    def update(self, name="", uid="", password="", dob=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        if len(dob) > 0:
            self.dob = dob
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

    def deleteGame(self, date):
        games = self.games.split('#')
        games.pop(0)
        for game in games:
            gameStr = game.replace("\'", "\"")
            thing = json.loads(gameStr)
            if thing['date'] == date:
                games.remove(game)
        gameString = ""
        for el in games:
            gameString += "#" + str(el)
        self.games = gameString
        db.session.commit()
        return gameString
            
    def make_gid(self):
        users = ChessUsers.query.all()
        gid = 0
        for user in users:
            if(user.get_gid() > gid):
                gid = user.get_gid()
        if (gid < 100):
            return 100
        return gid + 1

    # required for login_user, overrides id (login_user default) to implemented userID
    def get_gid(self):
        return self.gid
    
    def get_name(self):
        return self.name
